In [1]:
# !pip -q install git+https://github.com/huggingface/transformers # need to install from github
# !pip -q install datasets loralib sentencepiece 
# !pip -q install bitsandbytes accelerate

In [2]:
import json
import re

from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from notebook_util import NotebookUtil, Prompter

In [3]:
dataset_name = 'flowbird-april--flan-alpaca-gpt4-xl'

# 3B
# model_name = 'google/flan-t5-xl'
model_name = 'declare-lab/flan-alpaca-gpt4-xl' 

# 11B
# model_name = 'google/flan-t5-xxl'
# model_name = 'declare-lab/flan-alpaca-xxl'


# Init model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)


base_model = T5ForConditionalGeneration.from_pretrained(
# base_model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    # load_in_8bit=True,
    # device_map='auto'
).to("cuda")
# )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
!nvidia-smi

Sat Apr 22 23:31:59 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.47                 Driver Version: 531.68       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090         On | 00000000:01:00.0  On |                  N/A |
|  0%   48C    P2              139W / 350W|  13630MiB / 24576MiB |     25%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
question_prompter = Prompter(template_name="generate_question_t5_v2", runner="local")
answer_prompter = Prompter(template_name="generate_answer_t5_v2", runner="local")

dataset = NotebookUtil.load_dataset(dataset_name)

text_cases = NotebookUtil.load_raw_data_text()

[
    "WE ARE FLOWBIRD\nFlowbird offers end to end Smart mobility, Parking and Electric Vehicle charging solutions. We provide towns and cities with the tools to make mobility in urban areas simple, multimodal and environmentally friendly, and the ability to understand and manage this mobility.\nPartnering with local administrations and operators for over 65 years, we provide apps, terminals, kerbside management, enforcement - and the software to manage these remotely. We upgrade existing physical terminals to digital terminals with new services and tariffs and open them up to integrate third party apps. Flowbird orchestrates sales, payments, reporting, invoices and fines across multiple endpoints, also providing the data insights needed to inform decisions.\n",
    "\nFLOWBIRD global payment platform manages more than 300M banking transactions a year. Over 100 million users per week in 80+ countries trust Flowbird's expertise, R&D and technology partnerships to reduce air pollution, p

In [14]:
def process_case(text_case, dataset):
    q_prompt = question_prompter.generate_prompt(
        text_case,
        '3'
    )
    output = NotebookUtil.t5_generate(q_prompt, tokenizer, base_model)
    q_response = NotebookUtil.extract_response(output, model_type='flan')
    q_array = NotebookUtil.response_to_array(q_response, model_type='flan')


    a_prompt = answer_prompter.generate_prompt(
        text_case,
        '\n'.join(q_array)
    )
    output = NotebookUtil.t5_generate(a_prompt, tokenizer, base_model)
    a_response = NotebookUtil.extract_response(output, model_type='flan')
    a_array = NotebookUtil.response_to_array(a_response, model_type='flan')
    
    
    
    # NotebookUtil.print_wrap(q_prompt)
    # NotebookUtil.print_wrap( '\n'.join(q_array))

    print("\n\n---\n\n")

    NotebookUtil.print_wrap(a_prompt)
    # print(a_response)
    NotebookUtil.print_wrap( '\n'.join(a_array))
    
    NotebookUtil.question_answer_to_dataset(q_array, a_array, dataset)
    NotebookUtil.save_dataset(dataset_name, dataset)
    # NotebookUtil.print_json(dataset)

In [16]:
for text_case in text_cases:
  process_case(text_case, dataset)



---


### Instruction:
Below is a Description which describes a context. Use the context to answer all the questions.

### Description:
Within Flowbird. WE ARE FLOWBIRD
Flowbird offers end to end Smart mobility, Parking and Electric Vehicle charging solutions. We provide towns
and cities with the tools to make mobility in urban areas simple, multimodal and environmentally friendly, and
the ability to understand and manage this mobility.
Partnering with local administrations and operators for over 65 years, we provide apps, terminals, kerbside
management, enforcement - and the software to manage these remotely. We upgrade existing physical terminals to
digital terminals with new services and tariffs and open them up to integrate third party apps. Flowbird
orchestrates sales, payments, reporting, invoices and fines across multiple endpoints, also providing the data
insights needed to inform decisions.


### Questions:
1. What is the scope of Flowbird's Smart mobility solutions?
2. How 